# Find Threshold

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset, VideoData
from model.training_loop import train, EarlyStoppingParams
from model.multimodal_har_model import MultiModalHARModel

2025-11-27 22:29:37,785 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-11-27 22:29:37,785 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


### Model

In [2]:
OBSERVATION_RATIO = 100
EAR_RATIO = OBSERVATION_RATIO / 100
WITH_OBJECT_BRANCH = True

MODEL_PHT_PATH = "/Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v1.0.0_nw_ucla_20251124_153224.pht"

In [3]:
har_model, _ = MultiModalHARModel.load(
    checkpoint_path=MODEL_PHT_PATH
)

2025-11-27 22:29:49,355 - INFO - Loading model from /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v1.0.0_nw_ucla_20251124_153224.pht...
2025-11-27 22:29:49,861 - INFO - Model config: {'obj_in': 5, 'joint_in': 3, 'gat_hidden': 192, 'gat_out': 192, 'temporal_hidden': 192, 'num_classes': 10, 'dropout': 0.1, 'temporal_pooling': 'attn_pool', 'use_layer_norm': True, 'attention_pooling_heads': 4, 'temporal_transformer_heads': 4, 'use_object_branch': True, 'device': 'cpu'}
2025-11-27 22:29:49,864 - INFO - Model configuration: {'obj_in': 5, 'joint_in': 3, 'gat_hidden': 192, 'gat_out': 192, 'temporal_hidden': 192, 'num_classes': 10, 'dropout': 0.1, 'temporal_pooling': 'attn_pool', 'use_layer_norm': True, 'attention_pooling_heads': 4, 'temporal_transformer_heads': 4, 'use_object_branch': True, 'device': 'cpu'}
2025-11-27 22:29:50,042 - INFO - ✅ Model loaded and ready for inference


### Dataset

In [4]:
VALIDATION_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "test"
)
TEST_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "validation"
)

validation_video_data_loader = VideoDataLoader(
    path=VALIDATION_DIR,
)
test_video_data_loader = VideoDataLoader(
    path=TEST_DIR,
)


validation_dataset = VideoDataset(
    video_data_loader=validation_video_data_loader,
    normalization_type="across_frames",
    EAR_ratio=EAR_RATIO,
)

test_dataset = VideoDataset(
    video_data_loader=test_video_data_loader,
    normalization_type="across_frames",
    EAR_ratio=EAR_RATIO,
)

len(validation_dataset)
for _ in validation_dataset:
    pass

len(test_dataset)
for _ in test_dataset:
    pass

2025-11-27 22:29:50,106 - INFO - [VideoDataLoader] Loding action videos for action: a01


2025-11-27 22:29:50,184 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-11-27 22:29:50,279 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-11-27 22:29:50,303 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-11-27 22:29:50,335 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-11-27 22:29:50,376 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-11-27 22:29:50,401 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-11-27 22:29:50,722 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-11-27 22:29:50,829 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-11-27 22:29:50,857 - INFO - [VideoDataLoader] Loding action videos for action: a12
2025-11-27 22:29:51,441 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-11-27 22:29:51,476 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-11-27 22:29:51,515 - INFO -

## Finding Threshold

### Collecting Prediction + Labels

In [5]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve
from typing import cast

device = "cuda" if torch.cuda.is_available() else "cpu"
har_model = har_model.to(device).eval()

loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: x)

all_probs = []
all_labels = []

with torch.no_grad():
    for batch in loader:
        sample = cast(VideoData, batch[0])
        
        graphs_objects = [g.to(device) for g in sample.graphs_objects]  # adapt if needed
        graphs_joints = [g.to(device) for g in sample.graphs_joints]
        label = sample.label.unsqueeze(0).to(device)

        logits = har_model(graphs_objects, graphs_joints)
        probs = torch.softmax(logits, dim=1).cpu().numpy()   # shape: (N, num_classes)

        all_probs.append(probs)
        all_labels.append(label)

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)


In [6]:
print(all_probs.squeeze().shape)   # should be (147, 10)
print(all_labels.squeeze().shape)  # should be (147,)

(147, 10)
(147,)


### Search Optimal Threshold Using Youden’s J

In [7]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.preprocessing import label_binarize

def compute_youden_optimal_threshold_multiclass(all_probs, all_labels):
    """
    Computes optimal threshold per class using Youden's J statistic.
    Returns a dict: {class_index: optimal_threshold}
    """

    # Convert labels to 1D tensor (e.g., shape: (147,))
    all_labels = all_labels.squeeze()

    # Number of classes
    num_classes = all_probs.shape[-1]

    # Binarize labels for One-vs-Rest approach
    y_true_binarized = label_binarize(all_labels, classes=np.arange(num_classes))  
    # shape will be (N, num_classes)

    thresholds_per_class = {}

    for class_idx in range(num_classes):
        y_true = y_true_binarized[:, class_idx]  # binary ground-truth for THIS class
        y_score = all_probs[:, class_idx]        # probabilities for THIS class

        # ROC curve
        fpr, tpr, thresholds = roc_curve(y_true, y_score)

        # Youden's J statistic = TPR - FPR
        J_scores = tpr - fpr
        best_idx = np.argmax(J_scores)

        thresholds_per_class[class_idx] = thresholds[best_idx]

    return thresholds_per_class


thresholds = compute_youden_optimal_threshold_multiclass(all_probs.squeeze(), all_labels.squeeze())
print(thresholds)


{0: np.float32(0.04233511), 1: np.float32(0.40910658), 2: np.float32(0.019439017), 3: np.float32(0.032365076), 4: np.float32(0.76722306), 5: np.float32(0.6887442), 6: np.float32(0.1437972), 7: np.float32(0.023284385), 8: np.float32(0.032100104), 9: np.float32(0.6033996)}


### Testing Threshold

In [8]:
def evaluate_with_thresholds(model, dataset, thresholds):
    import torch
    import numpy as np

    device = 'cpu'
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            logits = model(graphs_objects, graphs_joints)
            probs = torch.softmax(logits, dim=1).cpu().numpy().flatten()   # shape: (num_classes,)

            # ---- APPLY THRESHOLD HERE ----
            adjusted = probs / np.array([thresholds[c] for c in range(len(probs))])
            predicted_class = np.argmax(adjusted)  # **single-class final decision**

            if predicted_class == label.item():
                correct += 1

            total += 1

    accuracy = 100 * correct / total
    return accuracy


In [9]:
accuracy = evaluate_with_thresholds(har_model, validation_dataset, thresholds)
print(f"Accuracy with class-wise threshold: {accuracy:.2f}%")


Accuracy with class-wise threshold: 77.55%


### Save for Inference